In [1]:
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET 
import numpy as np
import os
import random
import pickle

In [2]:
def save_pkl(data, filename):
    '''
    Saves any object as pickle file. Filename shall contain full path.
    '''
    with open(filename, 'wb') as output:
        pickle.dump(data, output)
        
def load_pkl(filename):
    '''
    Loads object from pickle file. Filename shall contain full path.
    '''
    with open(filename, 'rb') as input:
        return pickle.load(input)

In [3]:
# list of paths, where to take source photos
dataset_paths      = []
dataset_paths.append('/home/gosha20777/files/datasets/LADD_V4_winter/LizaAlertDroneDatasetV4_Winter')

# names of source folders in datasets
images_folder      =         'JPEGImages'
annotations_folder =         'Annotations'

# target folder - keeps resized X and y images
crops_folder       =         'segmentation_dataset'

# common crops details
initial_crop_size  =         512
crop_size          =         512
output_size        =         512

# random cropping details per image
random_crops       =         4

# target cropping details per image
target_crops       =         4
 

# target cropping details
files_per_dataset  =         10000


if crops_folder not in os.listdir():
    os.mkdir(Path(crops_folder))
    os.mkdir(Path(crops_folder, 'X'))
    os.mkdir(Path(crops_folder, 'y'))

In [4]:
train_indices    = []
val_indices      = []
test_indices     = []
trainval_indices = []

for dataset_path in dataset_paths:
    
    train = open(Path(dataset_path, 'ImageSets', 'Main', 'train.txt'))
    for line in train:
        train_indices.append(line.strip().replace('\n',''))  
    train.close()
    
    test = open(Path(dataset_path, 'ImageSets', 'Main', 'test.txt'))
    for line in test:
        test_indices.append(line.strip().replace('\n',''))  
    test.close()
    
    val = open(Path(dataset_path, 'ImageSets', 'Main', 'val.txt'))
    for line in val:
        val_indices.append(line.strip().replace('\n',''))  
    val.close()
    
    trainval = open(Path(dataset_path, 'ImageSets', 'Main', 'trainval.txt'))
    for line in trainval:
        trainval_indices.append(line.strip().replace('\n',''))  
    trainval.close()
    
save_pkl(train_indices, Path(crops_folder,'train_indices.pkl'))
save_pkl(val_indices, Path(crops_folder,  'val_indices.pkl'))
save_pkl(test_indices, Path(crops_folder, 'test_indices.pkl'))
save_pkl(trainval_indices, Path(crops_folder, 'trainval_indices.pkl'))

In [5]:
def get_crops():
    
    total_files_1 = 0
    for path in dataset_paths:
        total_files_1+=len(os.listdir(Path(path, images_folder)))
    total_files_2=files_per_dataset*len(dataset_paths)
        
    total_files = min(total_files_1, total_files_2)
    
    print('Generate target crops from %s files...' % total_files)
    
    n_crops = 0
    n_files = 0
    
    for dataset_path in dataset_paths:
        
        ds_files=0
        
        for filename in os.listdir(Path(dataset_path, annotations_folder)):
            
            if not filename.endswith('.xml'): continue
        
            fullname = Path(dataset_path, annotations_folder, filename)    
            tree = ET.parse(fullname)    
            root = tree.getroot()    
            bbox_num = 0
            
            img = cv2.imread(str(Path(dataset_path, images_folder, filename[:-3]+'jpg')))
            
            y = np.zeros_like((img)).astype('uint8')
            
            #### get target crops
            
            for rec in root:                
                if rec.tag == 'object':                    
                    for box in rec:
                        if box.tag=='bndbox':                            
                            # get initial bbox corners
                            ymin = int(box.findtext('ymin'))
                            ymax = int(box.findtext('ymax'))
                            xmin = int(box.findtext('xmin'))
                            xmax = int(box.findtext('xmax'))                            
                            y[ymin:ymax, xmin:xmax, :] = 255
                            
            for rec in root:
                    
                # get source image size
                if rec.tag == 'size': 
                    height = int(rec.findtext('height'))
                    width = int(rec.findtext('width'))
                    
                if rec.tag == 'object':
                    # list all available bboxes                        
                    for box in rec:
                        if box.tag=='bndbox':
                            
                            # get initial bbox corners
                            ymin = int(box.findtext('ymin'))
                            ymax = int(box.findtext('ymax'))
                            xmin = int(box.findtext('xmin'))
                            xmax = int(box.findtext('xmax'))
 
                            # calculate necessary padding to get crop of crop_size
                            padding_w = int((initial_crop_size - (xmax - xmin))/2.)
                            padding_h = int((initial_crop_size - (ymax - ymin))/2.)
            #############################################################################################
            
                            # get "target_crops" number of randomly shifted target crops
                            for cr in range(target_crops):
            
                                # get random shift within 25% of crop_size from bbox center
                                random_dx = int((random.random()-.5)*.5*initial_crop_size)
                                random_dy = int((random.random()-.5)*.5*initial_crop_size)

                                # calculate crop corners
                                new_xmin = xmin - padding_w + random_dx
                                new_xmax = xmax + padding_w + random_dx
                                new_ymin = ymin - padding_h + random_dy
                                new_ymax = ymax + padding_h + random_dy

                                # do not proceed if crop is outside of image
                                if (new_xmin<1 or new_xmax>width-1 or new_ymin<1 or new_ymax>height-1):continue

                                dx = new_xmax - new_xmin
                                dy = new_ymax - new_ymin

                                # correct crop corners to get exact crop_size
                                if dx<crop_size:
                                    if ((new_xmax+new_xmin)/2.)<(width/2.):
                                        new_xmax+=1
                                    else:
                                        new_xmin-=1
                                if dy<crop_size:
                                    if ((new_ymax+new_ymin)/2.)<(height/2.):
                                        new_ymax+=1
                                    else:
                                        new_ymin-=1

                                # create crop                            
                                X = img[new_ymin:new_ymax, new_xmin:new_xmax]                            
                                y_crop = y[new_ymin:new_ymax, new_xmin:new_xmax]

                                cv2.imwrite(str(Path(crops_folder, 'X', filename[:-4]+'___p'+format(n_crops,'05')+'.jpg')), X)
                                cv2.imwrite(str(Path(crops_folder, 'y', filename[:-4]+'___p'+format(n_crops,'05')+'.png')), y_crop)
                                n_crops+=1
 ######################################################################################################################

                            # goto next bbox in current file
                            bbox_num = bbox_num + 1  
        
#                             fig = plt.figure(figsize=(18,10))
#                             ax1 = fig.add_subplot('121')
#                             ax2 = fig.add_subplot('122')
#                             ax1.imshow(X)
#                             ax2.imshow(y_crop)
            
            #### get random crops
            
            for crop in range(random_crops):

                h0 = int(random.random()*(height - initial_crop_size))
                w0 = int(random.random()*(width - initial_crop_size))
                
                X = img[h0:h0+initial_crop_size, w0:w0+initial_crop_size]
                y_crop = y[h0:h0+initial_crop_size, w0:w0+initial_crop_size]
                
                if np.sum(y_crop)==0:
                    targ='n'
                else:
                    targ='p'
                
                cv2.imwrite(str(Path(crops_folder, 'X', filename[:-4]+'___'+targ+format(n_crops,'05')+'.jpg')), X)
                cv2.imwrite(str(Path(crops_folder, 'y', filename[:-4]+'___'+targ+format(n_crops,'05')+'.png')), y_crop)
                n_crops+=1
                
#                 fig = plt.figure(figsize=(18,10))
#                 ax1 = fig.add_subplot('121')
#                 ax2 = fig.add_subplot('122')
#                 ax1.imshow(X)
#                 ax2.imshow(y_crop)
        
            n_files+=1
            ds_files+=1
            
            if n_files in np.floor(np.linspace(0, total_files, 10)):print(np.int(n_files/total_files*100),'% passed...')
            if ds_files>files_per_dataset: break
                
    print(n_crops, 'crops created.')

In [6]:
get_crops()

Generate target crops from 411 files...
10 % passed...
22 % passed...
33 % passed...
44 % passed...
55 % passed...
66 % passed...
77 % passed...
88 % passed...
100 % passed...
6795 crops created.
